In [13]:
import pandas as pd
import numpy as np
import re

class PhoneFraudParser:
    def __init__(self, csv_path: str):
        self.csv_path = csv_path
        self.df_raw = None
        self.df_clean = None

    @staticmethod
    def extract(block, field):
        pattern = rf"{field}:(.*)"
        match = re.search(pattern, block)
        return match.group(1).strip() if match else ""

    @staticmethod
    def extract_price(price_str):
        if not price_str:
            return 0.0
        p = price_str.replace("lei", "").replace("Lei", "").replace(" ", "")
        p = re.sub(r"[^\d]", "", p)
        return float(p) if p.isdigit() else 0.0

    @staticmethod
    def extract_memory(text):
        if not isinstance(text, str):
            return 0
        m = re.search(r"(\d+)\s*gb", text.lower())
        return int(m.group(1)) if m else 0

    @staticmethod
    def extract_rating(text):
        m = re.search(r"([\d\.]+)\s*/\s*5", text)
        return float(m.group(1)) if m else 0.0

    @staticmethod
    def extract_num_ratings(text):
        m = re.search(r"(\d+)", text)
        return int(m.group(1)) if m else 0

    @staticmethod
    def parse_attributes(block):
        model = None
        condition = None

        if not isinstance(block, str):
            return model, condition

        parts = [p.strip() for p in block.split("|")]

        for p in parts:
            if "Model:" in p:
                model = p.replace("Model:", "").strip()
            if "Stare:" in p:
                condition = p.replace("Stare:", "").strip()

        return model, condition

    def load(self):
        self.df_raw = pd.read_csv(self.csv_path)
        return self

    def clean(self):
        rows = []

        for block in self.df_raw["Summary"]:
            title = self.extract(block, "Title")
            price_raw = self.extract(block, "Price")
            rating_raw = self.extract(block, "Rating")
            num_r_raw = self.extract(block, "Număr ratinguri")
            join_date = self.extract(block, "Cont OLX creat")
            photos_raw = self.extract(block, "Photos")
            description = self.extract(block, "Description")

            attr_match = re.search(r"Attributes:\s*\n(.*)", block)
            attributes_raw = attr_match.group(1).strip() if attr_match else ""
            model, condition = self.parse_attributes(attributes_raw)

            mem = self.extract_memory(title)
            if mem == 0:
                mem = self.extract_memory(description)

            rows.append({
                "phone_model": model,
                "memory_size": mem,
                "condition": condition,
                "number_of_photos": int(photos_raw) if photos_raw.isdigit() else 0,
                "publisher_rating": self.extract_rating(rating_raw),
                "publisher_num_ratings": self.extract_num_ratings(num_r_raw),
                "publisher_join_date": join_date,
                "clean_price": self.extract_price(price_raw)
            })

        df = pd.DataFrame(rows)

        df["median_price"] = (
            df.groupby(["phone_model", "memory_size", "condition"])["clean_price"]
              .transform(lambda g: g.replace(0, np.nan).median() or 0)
        )

        df["price_ratio"] = df.apply(
            lambda row: row["clean_price"] / row["median_price"]
                        if row["median_price"] > 0 else 1.0,
            axis=1
        )

        self.df_clean = df
        return self

    def get_clean_df(self):
        return self.df_clean


In [14]:
parser = PhoneFraudParser("/home/haidau_rares/projects/fraud_detection_iphones/olx_ads_data.csv")
parser.load().clean()

df_clean = parser.get_clean_df()
df_clean.to_csv("clean_olx_ads_data.csv", index=False)
df_clean.head()


/home/haidau_rares/projects/fraud_detection_iphones/venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,phone_model,memory_size,condition,number_of_photos,publisher_rating,publisher_num_ratings,publisher_join_date,clean_price,median_price,price_ratio
0,iPhone 1st gen,256,Utilizat,3,4.7,29,octombrie 2023,6500.0,4575.0,1.420765
1,iPhone 1st gen,512,Utilizat,2,5.0,55,iulie 2020,7300.0,7300.0,1.000000
2,iPhone 13,128,Utilizat,5,4.7,29,octombrie 2023,1600.0,1600.0,1.000000
3,iPhone 12 Mini,128,Utilizat,5,4.7,29,octombrie 2023,700.0,850.0,0.823529
4,iPhone 13 Pro,256,Utilizat,5,4.7,29,octombrie 2023,2100.0,2050.0,1.024390
